In [49]:
import pandas as pd
import numpy as np



users = pd.read_csv("data/users.csv")
movies = pd.read_csv("data/movies.csv")
ratings = pd.read_csv("data/ratings.csv")

# input to class are dataframes 
# few imports might be needed 

# class Recommender_System_Movies_DataSet
# and class function .get_recommendations(user_id,kthe user_id for which recommendation needed adn k being the the no. of recommendation needed 


Structure of code is as usch 

1. Recommender_System_Movies_DataSet Class takes input as dataframes for ratings, movies and users
   Once done two functions will automatically run to create attributes for the class one is 
   self.content_similarity_matrix this will have similarity matrix for content based filtering other 
   self.collabrative_model this function trains a model for matrix factorization for collabrative 

once this is trained we have content matrix and collabratie model for predictions 
 
2. once class object is created we can run function 
    get_recommendations(user_id: int, k: int)
    user_id being the user we need movies recommended for 
    k being the no. of movies 
    it computes both k movies for collabrative and content based approaches
    and that is input to hybrid function inside it which combines them on set of logic 
    
to get new recommendation again run get_recommendations for new user id or k value no need to train model 
is it gets trained with class initalization


NOTE: it takes time to train model hence Hence I decreased epochs to 2 to egt better prediction 
    at last line of collabrative_model  function change epochs =38 
    
    
    


In [73]:
import pandas
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam


class Recommender_System_Movies_DataSet():

    def __init__(self, ratings: pandas.DataFrame, movies: pandas.DataFrame,
                 users: pandas.DataFrame):


        self.movies  = movies
        self.ratings = ratings
        self.users    = users
        
        self.similarity_matrix= self.content_similarity_matrix(movies)
        self.model = self.collabrative_model(ratings)
        
    def content_similarity_matrix(self, movies):
        df=movies.copy() 
        df=df.set_index('movie_id')
        # setting movie id as index in case table has movie_ids not in order
        df=df.drop(columns = ['title','year'])
            
        # I am finding cosine similarity matrix 
        from sklearn.metrics.pairwise import cosine_similarity
        similarity_matrix = pd.DataFrame(cosine_similarity(df))
        
        return similarity_matrix
    
    
    def collabrative_model(self, ratings):
        # this part of code will take time to run 
        #each unique value is assigned a number from zeroi.e, category code
        ratings.user_id = ratings.user_id.astype('category').cat.codes.values
        ratings.movie_id = ratings.movie_id.astype('category').cat.codes.values

        #split the data set
        from sklearn.model_selection import train_test_split
        train, test = train_test_split(ratings, test_size=0.2)

        #number of users and movies
        n_users, n_movies = len(ratings.user_id.unique()), len(ratings.movie_id.unique())
        #create movie input of n movies  coulmn
        movie_input = keras.layers.Input(shape = [1], name = 'item')
        # create embedding vector space of n_movies 
        movie_embedding = keras.layers.Embedding(n_movies,  output_dim=50, input_length=1, name = 'movie_emb')(movie_input)
        #flatten into single vector 
        movie_vec = keras.layers.Flatten(name = "Flatten_Movies")(movie_embedding)


        user_input = keras.layers.Input(shape = [1], name = "User")
        user_vec = keras.layers.Flatten(name = 'Flatten_Users')(keras.layers.Embedding(n_users,  output_dim=50, input_length=1,
                                                                                        name = 'user_emb')(user_input))

        prod = keras.layers.dot([movie_vec, user_vec], axes = 1, name = "Dot_Product")

        model = keras.Model([user_input, movie_input], prod)
        model.compile(optimizer= Adam(learning_rate = 0.0005), loss='mean_squared_error', metrics=['mae', 'mse'])
        model.summary()
        
        #change epochs =38 below to get better result
        history = model.fit([train.user_id, train.movie_id], train.rating, batch_size=32, epochs=2, verbose=1, callbacks= EarlyStopping(patience=5,monitor="val_loss"),validation_data=([test.user_id, test.movie_id], test.rating) )
        """
            to display result graph use this 
            import matplotlib.pyplot as plt

            pd.Series(history.history['loss']).plot(logy=True)
            plt.xlabel("Epoch")
            plt.ylabel("Training Error")


        """
        
        return model
    
    
    
    
    def get_recommendations(self,user_id: int, k: int):
        
        """Recommends the k most similar movies of the movie with the id 'movie_id'.
        Parameters
        ----------
        movie_id :The id of the movie.
        k : int The number of similar movies to recommend.
        Returns: list of movie ids     
        """
        
        
        def content_based_filtering(user_id,k):
            similarity_matrix=self.similarity_matrix

            # finding movies liked by user with user_id so that recommendation can be made on those movies 
            # I am making one recommendation per 5 or 4 rated movie thus giving variation in reommnedation 

            movies_liked_by_user = ratings[ratings['user_id'] == user_id].sort_values(by='rating', ascending=False)
            movies_liked_by_user_list= list(movies_liked_by_user['movie_id'])
            
            
            # checking if we have 4 or 5 rated movies for that user 
            movies_liked_by_user_rated_4_or_5 = movies_liked_by_user[movies_liked_by_user['rating']>=4]
            if len(movies_liked_by_user_rated_4_or_5)<k:
                less_rating_avilable= True
            else:less_rating_avilable= False
                
            # if yes then we ill be recommending one movie from one rating rather than recommending more based on same movie
                
            
            Recommend= list()
            # this list will contain the movie ids we will recommend by this filtering method 
            
            if less_rating_avilable==False:
                for i in movies_liked_by_user_list:
                    while len(Recommend)!=k:
                        # finding the movie_ids that have highere cosine similarity 
                        
                        elements= similarity_matrix.loc[i].tolist()
            
                        index_numbers = np.argsort(elements).tolist()
                    
                        l = len(index_numbers)
                        
                        for j in range(l):
                            temp=index_numbers[-2-j] 
                            #since after agsoert the index with higher cosine similarity would be at end of list
                            # ignoring first one as it would be 1
                            
                            if (temp not in Recommend) and ( temp not in movies_liked_by_user) :
                                
                                            
                                """ 
                                The movies already present in rating table are watched by this user
                                hence we will not be including them here so if condition and also checking if 
                                the movie is not already recommended earlier

                               """
                            
                                Recommend.append(temp)
                                break
                            else: continue
                                
                                
            else:
                # for the case we don't have enough 4 or 5 rated movies
                while len(Recommend)!=k:
                    
             # basically if we have less high rated movies hence checking two movies per rated movie where rating <=3
            # we are recommending two movies per rated movie 
                    
                    movies_liked_by_user_above_3= list([movies_liked_by_user['rating']>=3]['movie_id'])
                    
                    # in case there are no ratings provided by user above 3
                    if len(movies_liked_by_user_above_3)==0: 
                        print( 'for the user_id {} not encough ratings given by user').format(user_id)
                    else:
                        for i in movies_liked_by_user_below_3:
                            elements= similarity_matrix.loc[i].tolist()
                            index_numbers = np.argsort(element).tolist()
                            l = len(index_numbers)
                            count=0
                            # counter used for every movie makes two recommendations as we don't have enough high ratings given by user
                            for j in range(l):
                                temp=index_numbers[-2-j] 
                                if temp not in Recommend and ( temp not in movies_liked_by_user):
                                    Recommend.append(temp)
                                    if count==2: break
                                else: continue

            
            return Recommend
        
        
        def collabrative_filltering(user_id,k):
            
            
            model= self.model
            #learnt embedding layer
            
            user_embedding_learnt = model.get_layer(name='user_emb').get_weights()[0]
            movie_embedding_learnt = model.get_layer(name='movie_emb').get_weights()[0]

            movies_now = user_embedding_learnt[user_id]@movie_embedding_learnt.T
            Recommend = list(np.argpartition(movies_now, -k)[-k:])

            return Recommend
        
        def Hybrid_Modelling(content, collabarative, movies, k):
            final=list()
            #first the movie_id common in both collabrative and content they would be taken
            for i in content:
                if i in collabarative:
                    final.append(i)
            if len(final)!=0 and len(final)!=k:
                for i in final:
                    content.remove(i)
                    collabarative.remove(i)


                for j in range(k-len(final)):
                    if len(final)!=k:
                        final.append(content[j])
                        if len(final)!=k:  
                            final.append(collabarative[j])
                        else: break
                    else: break


            elif len(final)==0:

                for j in range(k):
                    if len(final)!=k:
                        final.append(content[j])
                        if len(final)!=k:  
                            final.append(collabarative[j])
                        else: break
                    else: break

            df1 = pd.DataFrame( columns=['movie_id', 'title'])
            
            df2= pd.DataFrame( movies, columns=['movie_id', 'title']) 
            
            
            for i in final:
                
                temp= df2[df2['movie_id']==i]
                df1=df1.append(temp,ignore_index = True)
            return df1

        
       
        content_output= content_based_filtering(user_id,k)
        #print('content filtering output_movie_id is as following')
        #print(content_output)
        collabrative_output = collabrative_filltering(user_id,k)
        #print('collabrative filtering output_movie_id is as following')
        #print(collabrative_output)
        
        result= Hybrid_Modelling(content_output, collabrative_output, movies, k)
        
        print('combined result is')
        print(result)
        
        
        
        return
    
        
        
    
    
            
        
        

In [74]:
Object8= Recommender_System_Movies_DataSet(ratings, movies, users)

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
item (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
User (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
movie_emb (Embedding)           (None, 1, 50)        185300      item[0][0]                       
__________________________________________________________________________________________________
user_emb (Embedding)            (None, 1, 50)        302000      User[0][0]                       
___________________________________________________________________________________________

In [75]:
Object8.get_recommendations(12,20)

combined result is
   movie_id                                       title
0       242                           Glass Shield, The
1       843  Ballad of Narayama, The (Narayama Bushiko)
2      3714                                    Croupier
3       309                     Stuart Saves His Family
4      2199                             Few Good Men, A
5       837                         Spitfire Grill, The
6       427                               Carlito's Way
7      1170                                Passion Fish
8      1876                           On the Waterfront
9      1113                        Escape from New York
10     3160                        Another Man's Poison
11      253                                      Junior
12      280                            New Jersey Drive
13     1848                                  Armageddon
14     2162                                    Rounders
15     1108                              On Golden Pond
16      287                  